# Initialization

In [ ]:
# Parameters. 
# Times Square replaces this cell with the user's parameters.

# The run-time defaults for all of these parameters are in NightLog.yaml
# Under Times Square, the run-time defaults always override values given here.
# Values here are used for local tests.

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
day_obs = '2024-09-04' # Value to use for local testing (Summit)
#!day_obs = 'TODAY' # TODO Change to 'TODAY' to test with default before push  

# Total number of days of data to display (ending on day_obs)
number_of_days = '1'  # TODO Change to '1' to test with default before push  

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp, pformat
from urllib.parse import urlencode
from matplotlib import pyplot as plt
import os
from datetime import datetime, date, timedelta

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get('EXTERNAL_INSTANCE_URL'):
    print('Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch....')
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype >/dev/null
import lsst.ts.logging_and_reporting.source_adapters as sad
import lsst.ts.logging_and_reporting.almanac as alm
import lsst.ts.logging_and_reporting.reports as rep 
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md,mdlist

try:
    import lsst.ts.logging_and_reporting.version
    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = 'LIVE'

In [ ]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 500  # YAGNI for Auto get more if this isn't enough to get all requested DAYS

date = ut.get_datetime_from_day_obs_str(day_obs)
# date:  is EXLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_day_obs = (date - timedelta(days=days-1)).strftime('%Y-%m-%d') # Inclusive
max_day_obs = (date + timedelta(days=1)).strftime('%Y-%m-%d') # prep for Exclusive

response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

In [ ]:
# Set default env to "usdf" and try before PUSH to repo.
summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get('EXTERNAL_INSTANCE_URL', usdf) # TODO try with "usdf" before push (else "summit")

# Overview

The only environment that has everything needed for this page is
https://summit-lsp.lsst.codes

However, Times Square **does not** run on the Summit. It **does** run on USDF-dev.  USDF doesn't fully support all the data sources we need so some functionality is currently missing on this page.

In [ ]:
# Display overview of Report context 
md(f'''
Report for **{date}** covering the previous **{days}** observing night(s).
- Run on logs from **{server}/**
- Using *Prototype* Logging and Reporting Version: **{lrversion}**
''')

endpoint_urls_str = '\n- '.join(['',*sad.all_endpoints(server)])
md(f'This report will attempt to use the following log sources: {endpoint_urls_str}')

# DDV

In [ ]:
DDV = f"{server}/rubintv-dev/ddv/index.html" if 'summit' in server else f"{server}/rubintv/ddv/index.html"
md(f'Access DDV part of RubinTV: {DDV}')

# Almanac

In [ ]:
# Display various almanac values (for moon, sun)
rep.AlmanacReport().almanac_as_dataframe()

# Night Report

In [ ]:
# Get data from Night Report log. Display nightly Jira BLOCKS.
nr_adapter = sad.NightReportAdapter(server_url=server,
                                    limit=limit,
                                    min_day_obs=min_day_obs,
                                    max_day_obs=max_day_obs,)

status = nr_adapter.get_reports()
nr_url = status['endpoint_url']
#display(status)
rep.adapter_overview(nr_adapter, status, limit)

In [ ]:
# Display Jira BLOCKS
front = 'https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/'
tickets = nr_adapter.nightly_tickets(nr_adapter.records)

if tickets:
    mdstr = '## Nightly Jira BLOCKs'
    for day, url_list in tickets.items():
        mdstr += f'\n- {day}'
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    md(f'No jira BLOCK tickets found.', color='lightblue')
    md(f'Used: [API Data]({nr_url})')

# Display time log
nr_rep = rep.NightlyLogReport(min_day_obs=min_day_obs, max_day_obs=max_day_obs)
nr_rep.time_log_as_markdown(nr_adapter, nr_url)

# Exposure Log

In [ ]:
# Get data from Exposure log. Display time log.
exposure_adapter = sad.ExposurelogAdapter(
    server_url=server,
    limit=limit,
    min_day_obs=min_day_obs,
    max_day_obs=max_day_obs,
)
status = exposure_adapter.get_messages()
exposure_url = status['endpoint_url']
rep.adapter_overview(exposure_adapter, status, limit)

# Display time log
exposure_rep = rep.ExposurelogReport(min_day_obs=min_day_obs, max_day_obs=max_day_obs)
exposure_rep.time_log_as_markdown(exposure_adapter, exposure_url)

In [ ]:
# Display Observation gaps
gaps = exposure_adapter.get_observation_gaps()
if gaps:
    md(f'### Date vs Observation Gap (minutes) for all Instruments')
    for instrument, day_gaps in gaps.items():
        if len(day_gaps) == 0:
            md(f'**No day gaps found for *{instrument=!s}* **', color='lightblue')
        else:
            x,y = zip(*day_gaps.items())
            df = pd.DataFrame(dict(day=x,minutes=y))
            df.plot.bar(x='day', y='minutes', title=f'{instrument=!s}')

# Narrative Log


In [ ]:
# Get data from Narrative log. Display time log.
narrative_adapter = sad.NarrativelogAdapter(
    server_url=server,
    limit=limit,
    min_day_obs=min_day_obs,
    max_day_obs=max_day_obs,
)
status = narrative_adapter.get_messages()
narrative_url = status['endpoint_url']
rep.adapter_overview(narrative_adapter, status, limit)

narrrative_rep = rep.NarrativelogReport(min_day_obs=min_day_obs, max_day_obs=max_day_obs)
narrrative_rep.time_log_as_markdown(narrative_adapter, narrative_url)

# Developer Only Section

In [ ]:
# Conditionally display our current ability to connect to all needed endpoints.
if not os.environ.get('EXTERNAL_INSTANCE_URL'):
    md('## Dashboard')
    md('(This is not done when running under Times Square.)')
    %run ./dashboard.ipynb

# Finale

In [ ]:
print(f'Finished {str(datetime.now())}')